In [1]:
import pandas as pd
import textwrap
df_checkpoint = pd.read_csv('data/df_gme_sentiment_checkpoint_2.csv')

In [2]:
# Inspect a sentiment returned from the LLM.
text = df_checkpoint.iloc[-1]['sentiment']
wrapped_text = textwrap.fill(text, width=80)
print(wrapped_text)

You are a financial expert and your task is to determine whether a Reddit post
could potentially influence the stock market. Analyze the content of the post,
looking for mentions of specific stocks, price predictions, major news, or
significant sentiment. Answer 'yes' if the post is likely to have an impact and
'no' otherwise. Reddit post: ' so how long do you guys think this bubble will
last i am not a shill either i look at the bigger pictureeconomy is in bad shape
every state is in need of workers lots of people bout to get booted out of homes
people wanting more pay i see companies that are barely holding there head out
of the water look at gme for example the company has had plenty of time to do
something yet its same store all bag holders are trying to convince everyone to
jump back on last time i checked gme dont really do much for there employees any
ways how many of you put money into gme but never shop there i order my games
online for my fam its easier digital downloads no s

In [3]:
import re
df_checkpoint['sentiment_binary'] = 0
# Function to identify if 'yes' appears shortly after 'Response:'
def detect_yes(response):
    if pd.isna(response):  # Handle missing values
        return 0
    # Regex: Look for 'Response:' followed by anything, then 'yes' case-insensitively
    match = re.search(r"response:\s*.*?\byes\b", response, re.IGNORECASE)
    return 1 if match else 0

In [4]:
# Apply the function to create the new column
df_checkpoint['sentiment_binary'] = df_checkpoint['sentiment'].apply(detect_yes)

# Verify the results
print(df_checkpoint[['sentiment', 'sentiment_binary']].head())

                                           sentiment  sentiment_binary
0  You are a financial expert and your task is to...                 1
1  You are a financial expert and your task is to...                 1
2  You are a financial expert and your task is to...                 1
3  You are a financial expert and your task is to...                 1
4  You are a financial expert and your task is to...                 1


In [5]:
df_yes = df_checkpoint[df_checkpoint['sentiment_binary'] == 1]
df_yes

,title,score,id,comms_num,body,timestamp,extracted_tickers,gme,sentiment,sentiment_binary
0,exit the system,0,l6uhhn,47,the ceo of nasdaq pushed to halt trading to gi...,2021-01-28 21:30:35,gme,1,You are a financial expert and your task is to...,1
1,once youre done with gme $ag and $slv the gent...,0,l6u9wu,16,you guys are champs gme who would have thought...,2021-01-28 21:17:10,gme,1,You are a financial expert and your task is to...,1
2,my brokerage wants to force close my gme calls,560,l6te9s,215,my brokerage called me and asked me to close m...,2021-01-28 20:18:32,gme,1,You are a financial expert and your task is to...,1
3,rocketship to mars,215,l6tbye,90,holdbuy until gme marketcap is higher than th...,2021-01-28 20:14:23,gme,1,You are a financial expert and your task is to...,1
4,there is an alternative to this system,331,l6tauf,94,how fucking stupid is it that retail gets no c...,2021-01-28 20:12:07,gme,1,You are a financial expert and your task is to...,1
...,...,...,...,...,...,...,...,...,...,...
4641,ccl price target lets go cruising,41,ohl2zw,94,carnival cruise line $ccl you all may not hav...,2021-07-11 02:46:08,gme,1,You are a financial expert and your task is to...,1
4644,why lumen technologies is the value play,10,oyotn7,25,alright apes i know all of you are here to get...,2021-08-05 21:41:19,gme,1,You are a financial expert and your task is to...,1
4645,the hate on $hood is blinding people from maki...,0,oyfkfj,57,i get it robinhood is a scummy company that fu...,2021-08-05 14:27:33,gme,1,You are a financial expert and your task is to...,1
4646,unpopular opinion robinhood still has the best...,4829,ox50iz,1537,robinhood fucked me on gme and i moved all of ...,2021-08-03 17:48:44,gme,1,You are a financial expert and your task is to...,1


In [6]:
# Ensure the timestamp column is in datetime format
df_yes['timestamp'] = pd.to_datetime(df_yes['timestamp'])

# Find the earliest and latest posts
earliest_post = df_yes['timestamp'].min()
latest_post = df_yes['timestamp'].max()

print(f"Earliest Post: {earliest_post}")
print(f"Latest Post: {latest_post}")

Earliest Post: 2021-01-28 09:08:59
Latest Post: 2021-08-05 21:41:19


/var/folders/8d/y0sytz_d0h7bqr1lr6ywrfhr0000gn/T/ipykernel_16947/452532056.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yes['timestamp'] = pd.to_datetime(df_yes['timestamp'])


In [7]:
# Now we drop the columns: 'sentiment, gme, extracted_tickers, id. We could also drop the score, title and comms_num, but in all honesty, they are worth something.
# Just not sure I will use them.
# We also add combine the title and body, since information might be important in the title.

df_yes['post'] = df_yes['title'] + ' ' + df_yes['body']
df_yes = df_yes.drop(columns=['id', 'extracted_tickers', 'gme', 'sentiment', 'sentiment_binary', 'title', 'body'])

/var/folders/8d/y0sytz_d0h7bqr1lr6ywrfhr0000gn/T/ipykernel_16947/1252794492.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yes['post'] = df_yes['title'] + ' ' + df_yes['body']


In [8]:
#df_yes.to_csv('data/sentiment_ready_2.csv', sep=',', index=False)